In [7]:
from sqlalchemy.engine import URL
from sqlalchemy import create_engine, text, MetaData, Table

db_url = URL.create(
    drivername="mysql+pymysql",
    username="teamx",
    password="#C!D123^-c12",
    host="112.125.88.107",
    port=5906,
    database="TeamX_BIGAI",
    query={"charset": "utf8mb4"},
)

# db_url = URL.create(
#     drivername="mysql+pymysql",
#     username="agentictrl",
#     password="`1qaz~!QAZ",
#     host="112.125.88.107",
#     port=5906,
#     database="BIGAI",
#     query={"charset": "utf8mb4"},
# )

engine = create_engine(db_url, pool_pre_ping=True)

In [8]:

metadata = MetaData()

# 反射全部表结构
metadata.reflect(bind=engine)

print("数据库 BIGAI 包含以下表：")
for tbl in metadata.sorted_tables:
    print(f"\n🗂  表：{tbl.name}")
    for col in tbl.columns:
        print(f"   - {col.name} ({col.type}) {'PK' if col.primary_key else ''}")

数据库 BIGAI 包含以下表：

🗂  表：checkpoint
   - id (BIGINT) PK
   - name (VARCHAR(50)) 
   - version (VARCHAR(50)) 
   - run_id (VARCHAR(191)) 
   - status (VARCHAR(20)) 
   - path (VARCHAR(255)) 
   - source (VARCHAR(50)) 
   - operator (VARCHAR(50)) 
   - remark (VARCHAR(1024)) 
   - config_yaml (TEXT) 
   - created_at (TIMESTAMP) 
   - updated_at (TIMESTAMP) 
   - deleted_at (TIMESTAMP) 
   - started_at (TIMESTAMP) 
   - finished_at (TIMESTAMP) 

🗂  表：current_model
   - id (BIGINT) PK
   - checkpoint_id (BIGINT) 
   - version (VARCHAR(50)) 
   - path (VARCHAR(255)) 
   - status (VARCHAR(20)) 
   - created_at (TIMESTAMP) 
   - updated_at (TIMESTAMP) 
   - activated_by (VARCHAR(50)) 
   - remark (VARCHAR(1024)) 

🗂  表：dataset_usage_events
   - id (BIGINT) PK
   - trajectory_id (VARCHAR(128) COLLATE "utf8mb4_unicode_ci") 
   - run_id (VARCHAR(128) COLLATE "utf8mb4_unicode_ci") 
   - model_version (VARCHAR(512) COLLATE "utf8mb4_unicode_ci") 
   - used_delta (INTEGER) 
   - event_type (ENUM) 
   

In [ ]:
# run_id="results/pass8_20250919_trainset181_gpu4_env40_vllm_logp_maxstep30_osworld_new"
# run_id="results/ablation_novllm_logp_pass8_20250920_trainset45_gpu2_env20_vllm_logp_maxstep30_osworld_new"

# run_id="results/ablation_logp_pass8_20250920_trainset45_gpu2_env20_vllm_logp_maxstep30_osworld_new"
# run_id="results/test_trainset_26_plan"
run_id="results/trainset181_pass8_gpu4_env38_maxstep30_tmp1_osworldnew_20250918_1130"
with engine.connect() as conn:
    result = conn.execute(text(f"SELECT count(*) as count, avg(num_chunks) as avg_chunks, avg(reward) as avg_reward, count(distinct task_id) as unique_task_count, model_version FROM `rollout_run` where run_id='{run_id}' group by model_version ORDER BY min(id)"))
    rows = result.mappings().all()   # list[dict]，便于后续处理
print(f"rollout_run 共 {len(rows)} 行")


import pandas as pd
from IPython.display import display
# 可选：转成 DataFrame 方便查看
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(pd.DataFrame(rows))

rollout_run 共 0 行


""


In [ ]:
# results/pass16_20250901_train15_pass16_gpu2_env20_vllm_logp_maxstep15

# results/pass16_20250901_train15_pass16_gpu2_env20_vllm_logp_maxstep15

In [10]:
import pandas as pd

# 查看run_id已采样的数据
# run_id="results/test_for_train_pass8_gpu7_env69_20250821_2333" # results/test_for_train_instruction  results/pass@32_trainset90
# run_id="results/pass8_20250913_trainset152_gpu6_env58_vllm_logp_maxstep30_node2" # results/test_for_train_instruction  results/pass@32_trainset90
run_id="results/trainset181_pass8_gpu4_env38_maxstep30_tmp1_osworldnew_20250918_1130"
with engine.connect() as conn:
    result = conn.execute(text(f"SELECT * FROM `rollout_run` where run_id='{run_id}' order by create_at desc"))
    rows = result.mappings().all()   # list[dict]，便于后续处理
print(f"rollout_run 共 {len(rows)} 行")
# 可选：转成 DataFrame 方便查看
df = pd.DataFrame(rows)
if "task_id" in df.columns and "reward" in df.columns:
    avg_rewards = df.groupby("task_id")["reward"].mean()
    traj_counts = df.groupby("task_id")["trajectory_id"].nunique()
    summary_df = pd.DataFrame({
        "avg_reward": avg_rewards,
        "traj_count": traj_counts
    })
    print("每个 task 的统计：")
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(summary_df)
else:
    print("数据中缺少 task_id 或 reward 字段")


import pandas as pd; pd.DataFrame(rows)


rollout_run 共 19257 行
每个 task 的统计：


,avg_reward,traj_count
task_id,,
00fa164e-2612-4439-992e-157d019a8436,0.000000,104
02ce9a50-7af2-47ed-8596-af0c230501f8,0.138298,94
030eeff7-b492-4218-b312-701ec99ee0cc,0.589286,112
0512bb38-d531-4acf-9e7e-0add90816068,0.398058,103
06ca5602-62ca-47f6-ad4f-da151cde54cc,0.663265,98
06fe7178-4491-4589-810f-2e2bc9502122,0.946429,112
0810415c-bde4-4443-9047-d5f70165a697,0.901786,112
08aced46-45a2-48d7-993b-ed3fb5b32302,1.000000,112
09a37c51-e625-49f4-a514-20a773797a8a,0.104297,118


,create_at,id,instruction,model_version,num_chunks,reward,run_id,split_dir,task_id,trace_id,trajectory_id,used
0,2025-09-21 10:51:23,926889,Could you help me export an Impress file to a ...,/workspace/computer-use/verl/checkpoints/verl_...,9,0.903029,results/trainset181_pass8_gpu4_env38_maxstep30...,None,455d3c66-7dc6-4537-a39a-36d3e9119df7,trace-6bb96231f9dc-1758419899,455d3c66-7dc6-4537-a39a-36d3e9119df7_trace-6bb...,0
1,2025-09-21 10:51:23,926890,Could you help me export an Impress file to a ...,/workspace/computer-use/verl/checkpoints/verl_...,9,0.903029,results/trainset181_pass8_gpu4_env38_maxstep30...,None,455d3c66-7dc6-4537-a39a-36d3e9119df7,trace-b5e3686810d3-1758419899,455d3c66-7dc6-4537-a39a-36d3e9119df7_trace-b5e...,0
2,2025-09-21 10:51:19,926887,Could you help me export an Impress file to a ...,/workspace/computer-use/verl/checkpoints/verl_...,9,0.903029,results/trainset181_pass8_gpu4_env38_maxstep30...,None,455d3c66-7dc6-4537-a39a-36d3e9119df7,trace-bef351c21e5a-1758419899,455d3c66-7dc6-4537-a39a-36d3e9119df7_trace-bef...,0
3,2025-09-21 10:51:17,926885,"Copy the ""Revenue"" column along with the heade...",/workspace/computer-use/verl/checkpoints/verl_...,8,1.000000,results/trainset181_pass8_gpu4_env38_maxstep30...,None,1273e544-688f-496b-8d89-3e0f40aa0606,trace-f439c757020e-1758419899,1273e544-688f-496b-8d89-3e0f40aa0606_trace-f43...,0
4,2025-09-21 10:51:16,926884,"Copy the ""Revenue"" column along with the heade...",/workspace/computer-use/verl/checkpoints/verl_...,8,1.000000,results/trainset181_pass8_gpu4_env38_maxstep30...,None,1273e544-688f-496b-8d89-3e0f40aa0606,trace-06f2c4c90fac-1758419899,1273e544-688f-496b-8d89-3e0f40aa0606_trace-06f...,0
...,...,...,...,...,...,...,...,...,...,...,...,...
19252,2025-09-20 10:28:27,884748,Compare iPhone 15 Pro Max with iPhone 14 Pro M...,/workspace/computer-use/verl/checkpoints/verl_...,25,0.000000,results/trainset181_pass8_gpu4_env38_maxstep30...,None,f5d96daf-83a8-4c86-9686-bada31fc66ab,trace-225ecc412bf8-1758332261,f5d96daf-83a8-4c86-9686-bada31fc66ab_trace-225...,0
19253,2025-09-20 10:28:12,884746,Copies file 'file1' to each of directories 'di...,/workspace/computer-use/verl/checkpoints/verl_...,17,0.000000,results/trainset181_pass8_gpu4_env38_maxstep30...,None,6f56bf42-85b8-4fbb-8e06-6c44960184ba,trace-666c71de8509-1758332261,6f56bf42-85b8-4fbb-8e06-6c44960184ba_trace-666...,0
19254,2025-09-20 10:28:08,884745,Can you help me clean up my computer by gettin...,/workspace/computer-use/verl/checkpoints/verl_...,4,1.000000,results/trainset181_pass8_gpu4_env38_maxstep30...,None,7b6c7e24-c58a-49fc-a5bb-d57b80e5b4c3,trace-f7536abeb61b-1758332261,7b6c7e24-c58a-49fc-a5bb-d57b80e5b4c3_trace-f75...,0
19255,2025-09-20 10:28:05,884741,Copies file 'file1' to each of directories 'di...,/workspace/computer-use/verl/checkpoints/verl_...,15,1.000000,results/trainset181_pass8_gpu4_env38_maxstep30...,None,6f56bf42-85b8-4fbb-8e06-6c44960184ba,trace-036ffb36d56f-1758332261,6f56bf42-85b8-4fbb-8e06-6c44960184ba_trace-036...,0


In [ ]:
results/trainset181_pass8_gpu4_env38_maxstep30_tmp1_osworldnew_20250918_1130



In [8]:
# 查看run_id已保存的checkpoints
run_id="results/pass16_20250902_train90_pass16_gpu2_env20_vllm_logp_maxstep50" # results/test_for_train_instruction  results/pass@32_trainset90
with engine.connect() as conn:
    result = conn.execute(text("SELECT * FROM `checkpoint` where run_id = '{run_id}' "))
    rows = result.mappings().all()   # list[dict]，便于后续处理
print(f"checkpoint 共 {len(rows)} 行")
# 可选：转成 DataFrame 方便查看
print(rows[0])
import pandas as pd; pd.DataFrame(rows).head()

checkpoint 共 0 行


IndexError: list index out of range


# 从rollout_run中删除某run_id的数据
run_id="results/test_for_train_pass8_gpu7_env69_20250821_2333"
with engine.begin() as conn:
    will = conn.execute(
        text("SELECT COUNT(*) FROM `rollout_run` WHERE `run_id` = :rid"),
        {"rid": run_id}
    ).scalar_one()
    print(f"将删除 {will} 行（run_id = {run_id}）")

    res = conn.execute(
        text("DELETE FROM `rollout_run` WHERE `run_id` = :rid"),
        {"rid": run_id}
    )
    print(f"✅ 已删除 {res.rowcount} 行")

In [19]:
# run_id="results/test_for_train_pass8_gpu8_env77_20250817_1345"
with engine.connect() as conn:
    cnt = conn.execute(
        text("""
            SELECT COUNT(*) FROM `rollout_run`
            WHERE `run_id` = :{run_id} AND `used` != 0
        """),
        {"rid": run_id},
    ).scalar_one()
print(f"确认 used!=0 的行数：{cnt}")

with engine.begin() as conn:
    res = conn.execute(
        text("""
            UPDATE `rollout_run`
            SET `used` = 0
            WHERE `run_id` = :{run_id}
        """),
        {"rid": run_id},
    )
    print(f"✅ 更新行数: {res.rowcount}")

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near ':{run_id} AND `used` != 0' at line 2")
[SQL: 
            SELECT COUNT(*) FROM `rollout_run`
            WHERE `run_id` = :{run_id} AND `used` != 0
        ]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
# 更新used = 0 数据的model version
run_id="results/pass@32_trainset90"
model_version = "/workspace/computer-use/verl/checkpoints/verl_osworld_grpo/osworld_all_feasible_reward_script_grpo_k8s_0813_h8zdohoq/global_step_10/actor/huggingface"
model_version = "/capacity/userdata/vcfenxd75jiv/shichenrui/ui_tars/ByteDance-Seed/UI-TARS-1.5"
with engine.connect() as conn:
    cnt, sample = conn.execute(
        text("""
            SELECT COUNT(*) AS cnt, MAX(`model_version`) AS sample_version
            FROM `rollout_run`
            WHERE `run_id` = :rid AND `used` = 0
        """),
        {"rid": run_id},
    ).one()
print(f"used=0 的剩余行数: {cnt}，示例版本: {sample}")

with engine.begin() as conn:
    res = conn.execute(
        text("""
            UPDATE `rollout_run`
            SET `model_version` = :mv
            WHERE `run_id` = :rid AND `used` = 0
        """),
        {"mv": model_version, "rid": run_id},
    )
    print(f"✅ 更新行数: {res.rowcount}")

In [ ]:
# 统计run_id validation 结果

import numpy as np
import json
import pandas as pd
# val_trainset90_px_08220031_step10
# run_ids = ["results/val_train150_uitars_maxstep50", 
#            "results/val_trainset90_px_08220031_step10", 
#            "results/val_trainset90_px_08220031_step20", 
#            "results/val_trainset90_px_08220031_step30", 
#            "results/val_trainset90_px_08220031_step35",
#            "results/val_trainset90_px_08220031_step40"]
run_ids = ["results/pass16_20250825_train15_pass16_gpu4_env36_kl"]

with open("../evaluation_examples/test_trainset_15.json", "r", encoding="utf-8") as f:
    mapping = json.load(f)
eval_task_ids = set()
for _, lst in mapping.items():
    eval_task_ids.update(lst)
    
placeholders = ", ".join([f":id{i}" for i in range(len(run_ids))]) or ":id0"
sql = text(f"""
    SELECT reward, run_id, task_id
    FROM rollout_run
    WHERE run_id IN ({placeholders})
""")
params = {f"id{i}": rid for i, rid in enumerate(run_ids)} or {"id0": None}

with engine.begin() as conn:
    df = pd.read_sql(sql, conn, params=params)

def agg_by_run_id(df: pd.DataFrame) -> pd.DataFrame:
    x = df["reward"].fillna(0)
    reward_non_neg1 = np.where(x.eq(-1), 0, x).astype(float)

    df_calc = pd.DataFrame({
        "run_id": df["run_id"].values,
        "reward_non_neg1": reward_non_neg1
    })

    total_rows = df_calc.groupby("run_id").size().rename("total_rows")
    total_score = df_calc.groupby("run_id")["reward_non_neg1"].sum().rename("total_score")

    result = (
        pd.concat([total_rows, total_score], axis=1)
        .assign(success_rate=lambda d: d["total_score"] / d["total_rows"])
        .reset_index()
        .sort_values("run_id")
    )
    return result

df_subset = df[df["task_id"].isin(eval_task_ids)]
result_subset = agg_by_run_id(df).rename(columns={
    "metric": "metric_subset",
    "total_score": "total_score",
    "total_rows": "total_rows_subset",
})

result_subset

In [22]:
result_subset = agg_by_run_id(df_subset).rename(columns={
    "metric": "metric_subset",
    "total_score": "total_score",
    "total_rows": "total_rows_subset",
})
result_subset

NameError: name 'agg_by_run_id' is not defined